In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
%fs
ls /Volumes/workspace/ecommerce/ecommerce_data

path,name,size,modificationTime
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv,2019-Nov.csv,9006762395,1767962633000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv,2019-Oct.csv,5668612855,1767962553000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/delta/,delta/,0,1768292740448
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/incremental/,incremental/,0,1768292740448
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/parquet/,parquet/,0,1768292740449


In [0]:
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")

In [0]:
df_oct.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
print("Total records:", df_oct.count())

Total records: 42448764


#### In modern Databricks (with Unity Catalog):

#### Public DBFS root ( /delta, /dbfs, /FileStore, etc.) is DISABLED

#### You cannot write directly to /delta/...

#### You must write to Unity Catalog locations, i.e. Volumes or Managed Tables

#### designed for security & governance.

In [0]:
# Convert to Delta
df_oct.write.format("delta").mode("overwrite").save("/delta/events")

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-5684726801837079>, line 2
      1 # Convert to Delta
----> 2 df_oct.write.format("delta").mode("overwrite").save("/delta/events")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:703, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    701     self.format(format)
    702 self._write.path = path
--> 703 _, _, ei = self._spark.client.execute_command(
    704     self._write.command(self._spark.client), self._write.observations
    705 )
    706 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_id
   1555 req.plan.command.CopyFrom(command)
-> 1556 data, _, metr

In [0]:
df_oct.write.format("delta").mode("overwrite").save("/Volumes/workspace/ecommerce/ecommerce_data/delta/events_oct")


In [0]:
# Create managed table
df_oct.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce.events_oct")

In [0]:
%sql
DESCRIBE DETAIL workspace.ecommerce.events_oct;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,e088ce1a-47f4-4ec7-9f43-7dfd0d8deb31,workspace.ecommerce.events_oct,null,,2026-01-12T07:15:56.908Z,2026-01-13T08:26:41.000Z,List(),List(),12,1471926065,Map(delta.enableDeletionVectors -> true),3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
spark.sql("""
CREATE TABLE workspace.ecommerce.events_delta USING DELTA
AS
SELECT *
FROM workspace.ecommerce.events_oct
""")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5684726801837083>, line 1
----> 1 spark.sql("""
      2 CREATE TABLE workspace.ecommerce.events_delta USING DELTA
      3 AS
      4 SELECT *
      5 FROM workspace.ecommerce.events_oct
      6 """)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:875, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    872         _views.append(SubqueryAlias(df._plan, name))
    874 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 875 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    876 if "sql_command_result" in properties:
    877     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations

### SCHEMA ENFORCEMENT
#### I design pipelines with schema enforcement, schema evolution, and data quality checks using Delta Lake.
#### Delta Lake will NOT allow data that does not match the table’s schema to be written.
#### No silent corruption. No guessing. No bad data.
#### Schema enforcement - Block wrong data.
#### Schema evolution - Allow controlled new columns


#### Schema enforcement == data firewall. We test it to make sure garbage never enters our system.

In [0]:
try:
    wrong_schema = spark.createDataFrame(
        [("a", "b", "c")],
        ["x", "y", "z"]
    )

    wrong_schema.write.format("delta") \
        .mode("append") \
        .saveAsTable("workspace.ecommerce.events_oct")

except Exception as e:
    print("Schema enforcement triggered:")
    print(e)

Schema enforcement triggered:
[_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: e088ce1a-47f4-4ec7-9f43-7dfd0d8deb31).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- product_id: integer (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nullable = true)
-- brand: string (nullable = true)
-- price: double (nullable = true)
-- user_id: integer (nullable = true)
-- user_session: string (nullable = true)
-- source_system: string (nullable = true)


Data schema:
root
-- x: string (nullable = true)
-- y: string (nullable = true)
-- z: string (nullable = true)

        

#### This confirms schema enforcement is working correctly in your Delta + Unity Catalog setup.

In [0]:
from pyspark.sql.functions import lit
df_new_col = df_oct.withColumn("source_system", lit("web"))

In [0]:
df_new_col.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- source_system: string (nullable = false)



In [0]:
df_new_col.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("workspace.ecommerce.events_oct")

In [0]:
%sql
DESCRIBE workspace.ecommerce.events_oct;

col_name,data_type,comment
event_time,timestamp,null
event_type,string,null
product_id,int,null
category_id,bigint,null
category_code,string,null
brand,string,null
price,double,null
user_id,int,null
user_session,string,null
source_system,string,null
